## Project 1 - Los Angeles Traffic Collision Analysis

In [1]:
# import modules for the analysis environment
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# read csv file
df = pd.read_csv("Data/Raw/traffic-collision-data-from-2010-to-present.csv")
df.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Premise Description,Address,Cross Street,Location,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified)
0,190614534,2019-06-29T00:00:00.000,2019-06-29T00:00:00.000,1000,6,Hollywood,626,997,TRAFFIC COLLISION,NaN,...,STREET,PRIMROSE AV,VINE ST,"{'latitude': '34.1098', 'longitude': '-118.326...",24033.0,405.0,371.0,NaN,7.0,64.0
1,190411644,2019-06-29T00:00:00.000,2019-06-29T00:00:00.000,710,4,Hollenbeck,479,997,TRAFFIC COLLISION,NaN,...,STREET,WHITTIER BL,SPENCE ST,"{'latitude': '34.0265', 'longitude': '-118.198...",23441.0,527.0,352.0,NaN,9.0,38.0
2,190913064,2019-06-29T00:00:00.000,2019-06-29T00:00:00.000,710,9,Van Nuys,923,997,TRAFFIC COLLISION,NaN,...,STREET,VAN NUYS BL,VANOWEN ST,"{'latitude': '34.1939', 'longitude': '-118.448...",19733.0,236.0,290.0,NaN,3.0,18.0
3,191217109,2019-06-29T00:00:00.000,2019-06-29T00:00:00.000,125,12,77th Street,1245,997,TRAFFIC COLLISION,4025 3036 3004 3026 3029 3101,...,STREET,VERMONT AV,FLORENCE AV,"{'latitude': '33.9746', 'longitude': '-118.291...",23675.0,777.0,1161.0,7.0,14.0,20.0
4,191011660,2019-06-29T00:00:00.000,2019-06-29T00:00:00.000,40,10,West Valley,1005,997,TRAFFIC COLLISION,NaN,...,STREET,SATICOY ST,BALCOM AV,"{'latitude': '34.2084', 'longitude': '-118.520...",18909.0,253.0,914.0,NaN,2.0,12.0


## Stats of Series in the Dataset

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478415 entries, 0 to 478414
Data columns (total 24 columns):
DR Number                            478415 non-null int64
Date Reported                        478415 non-null object
Date Occurred                        478415 non-null object
Time Occurred                        478415 non-null int64
Area ID                              478415 non-null int64
Area Name                            478415 non-null object
Reporting District                   478415 non-null int64
Crime Code                           478415 non-null int64
Crime Code Description               478415 non-null object
MO Codes                             393593 non-null object
Victim Age                           400964 non-null float64
Victim Sex                           471443 non-null object
Victim Descent                       470724 non-null object
Premise Code                         478390 non-null float64
Premise Description                  478390 non-null

## Cleaning Data

In [3]:
dfclean = df.drop(columns=['DR Number', #unique value but we have index
                           'Date Reported', #repetitive series
                           'MO Codes', #mostly NaN values
                           'Crime Code', #repetitive
                           'Crime Code Description', #uniform data
                           'LA Specific Plans', #unrelated data with NaN values
                           'Area ID', #repetitive data
                           'Premise Description',#useless data 
                           'Neighborhood Councils (Certified)', #irrelevant
                           'Precinct Boundaries', #irrelevant
                           'Council Districts', # irrelevant
                           'Census Tracts']) #meaningless data
dfclean.head()

,Date Occurred,Time Occurred,Area Name,Reporting District,Victim Age,Victim Sex,Victim Descent,Premise Code,Address,Cross Street,Location,Zip Codes
0,2019-06-29T00:00:00.000,1000,Hollywood,626,NaN,M,W,101.0,PRIMROSE AV,VINE ST,"{'latitude': '34.1098', 'longitude': '-118.326...",24033.0
1,2019-06-29T00:00:00.000,710,Hollenbeck,479,25.0,M,H,101.0,WHITTIER BL,SPENCE ST,"{'latitude': '34.0265', 'longitude': '-118.198...",23441.0
2,2019-06-29T00:00:00.000,710,Van Nuys,923,35.0,M,H,101.0,VAN NUYS BL,VANOWEN ST,"{'latitude': '34.1939', 'longitude': '-118.448...",19733.0
3,2019-06-29T00:00:00.000,125,77th Street,1245,34.0,M,B,101.0,VERMONT AV,FLORENCE AV,"{'latitude': '33.9746', 'longitude': '-118.291...",23675.0
4,2019-06-29T00:00:00.000,40,West Valley,1005,50.0,F,H,101.0,SATICOY ST,BALCOM AV,"{'latitude': '34.2084', 'longitude': '-118.520...",18909.0


In [4]:
dfclean.columns = ['date','time','area','district','victim_age','victim_sex','victim_race',
                   'premise','address','cross_street','location','zip']
dfclean.head()

,date,time,area,district,victim_age,victim_sex,victim_race,premise,address,cross_street,location,zip
0,2019-06-29T00:00:00.000,1000,Hollywood,626,NaN,M,W,101.0,PRIMROSE AV,VINE ST,"{'latitude': '34.1098', 'longitude': '-118.326...",24033.0
1,2019-06-29T00:00:00.000,710,Hollenbeck,479,25.0,M,H,101.0,WHITTIER BL,SPENCE ST,"{'latitude': '34.0265', 'longitude': '-118.198...",23441.0
2,2019-06-29T00:00:00.000,710,Van Nuys,923,35.0,M,H,101.0,VAN NUYS BL,VANOWEN ST,"{'latitude': '34.1939', 'longitude': '-118.448...",19733.0
3,2019-06-29T00:00:00.000,125,77th Street,1245,34.0,M,B,101.0,VERMONT AV,FLORENCE AV,"{'latitude': '33.9746', 'longitude': '-118.291...",23675.0
4,2019-06-29T00:00:00.000,40,West Valley,1005,50.0,F,H,101.0,SATICOY ST,BALCOM AV,"{'latitude': '34.2084', 'longitude': '-118.520...",18909.0


## victim_race

In [5]:
race_description = {'H':'Hispanic', 'B':'Black', 'O':'Unknown', 'W':'White', 'X':'Unknown', '-':'Unknown',
                    'A':'Asian', 'K':'Asian', 'C':'Asian', 'F':'Asian', 'U':'Pacific Islander',
                    'J':'Asian', 'P':'Pacific Islander', 'V':'Asian', 'Z':'Asian',
                    'I':'American Indian', 'G':'Pacific Islander', 'S':'Pacific Islander', 
                    'D':'Asian', 'L':'Asian'}
dfclean.victim_race = dfclean.victim_race.map(race_description)
dfclean.head()

,date,time,area,district,victim_age,victim_sex,victim_race,premise,address,cross_street,location,zip
0,2019-06-29T00:00:00.000,1000,Hollywood,626,NaN,M,White,101.0,PRIMROSE AV,VINE ST,"{'latitude': '34.1098', 'longitude': '-118.326...",24033.0
1,2019-06-29T00:00:00.000,710,Hollenbeck,479,25.0,M,Hispanic,101.0,WHITTIER BL,SPENCE ST,"{'latitude': '34.0265', 'longitude': '-118.198...",23441.0
2,2019-06-29T00:00:00.000,710,Van Nuys,923,35.0,M,Hispanic,101.0,VAN NUYS BL,VANOWEN ST,"{'latitude': '34.1939', 'longitude': '-118.448...",19733.0
3,2019-06-29T00:00:00.000,125,77th Street,1245,34.0,M,Black,101.0,VERMONT AV,FLORENCE AV,"{'latitude': '33.9746', 'longitude': '-118.291...",23675.0
4,2019-06-29T00:00:00.000,40,West Valley,1005,50.0,F,Hispanic,101.0,SATICOY ST,BALCOM AV,"{'latitude': '34.2084', 'longitude': '-118.520...",18909.0


## Longitude and Latitude

In [6]:
import ast

In [7]:
dfclean['location'] = [ast.literal_eval(d) for d in dfclean.location]
dfclean['longitude'] = [d['longitude'] for d in dfclean.location]
dfclean['latitude'] = [d['latitude'] for d in dfclean.location]
dfclean.longitude = dfclean.longitude.astype(float)
dfclean.latitude = dfclean.latitude.astype(float)
dfclean.head()

,date,time,area,district,victim_age,victim_sex,victim_race,premise,address,cross_street,location,zip,longitude,latitude
0,2019-06-29T00:00:00.000,1000,Hollywood,626,NaN,M,White,101.0,PRIMROSE AV,VINE ST,"{'latitude': '34.1098', 'longitude': '-118.326...",24033.0,-118.3267,34.1098
1,2019-06-29T00:00:00.000,710,Hollenbeck,479,25.0,M,Hispanic,101.0,WHITTIER BL,SPENCE ST,"{'latitude': '34.0265', 'longitude': '-118.198...",23441.0,-118.1980,34.0265
2,2019-06-29T00:00:00.000,710,Van Nuys,923,35.0,M,Hispanic,101.0,VAN NUYS BL,VANOWEN ST,"{'latitude': '34.1939', 'longitude': '-118.448...",19733.0,-118.4487,34.1939
3,2019-06-29T00:00:00.000,125,77th Street,1245,34.0,M,Black,101.0,VERMONT AV,FLORENCE AV,"{'latitude': '33.9746', 'longitude': '-118.291...",23675.0,-118.2918,33.9746
4,2019-06-29T00:00:00.000,40,West Valley,1005,50.0,F,Hispanic,101.0,SATICOY ST,BALCOM AV,"{'latitude': '34.2084', 'longitude': '-118.520...",18909.0,-118.5208,34.2084


In [10]:
dfclean = dfclean.drop(columns=['location'])
dfclean.to_csv('Data/Clean/cleaned_data.csv',index=False)